In [2]:
from manim import *
from numpy import linalg as npl
import jupyter_capture_output
from scipy.interpolate import make_interp_spline, BSpline

video_scene = " -v WARNING --progress_bar None --disable_caching et_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s et_Scene"

Manim Community v0.16.0.post0

Jupyter Capture Output v0.0.6


In [54]:
bin_count = 8

# Array mit Eye-Tracking Daten ([x, y, t])
et_array = np.array([
    [-6, -2, 200],
    [-4, 0, 400],
    [-3, -1, 600],
    [-2.1, 2, 200],
    [-4, 3, 100],
    [-5, 2, 100],
    [-4.3, 2.4, 100],
    [-2.5, 1, 100],
    [-2, -1, 300],
    [-1.7, 2, 100],
    [-3, 3.3, 100],
    [-2.8, 3, 100],
    [-1, 2.5, 100],
    [-1, -3, 100],
    [-2.3, -3, 100],
    [-2, -2.2, 200],
    [-3, -3, 500],
    [-4.1, -3.2, 200],
    [-2.1, -2.7, 100],
    [-3, -1, 600],
    [-2.5, 2, 200],
    [-4, 3, 100],
    [-5, 2, 100],
    [-4.3, 2.4, 100],
    [-1.5, 1, 100],
    [-2, -1, 300],
    [-1.7, 2, 100],
    [-2.1, 3.1, 100],
    [-2.8, 3, 100],
    [-2.3, 2.5, 100],
    [-1, -3.25, 100],
    [-2.3, -1, 100],
    [-1, -1.2, 200],
    [-3, -3, 500],
    [-4.7, -3.2, 200],
    [-2.1, -2.7, 100],
    [-4, 3, 100],
    [-5, 2, 100],
    [-2.3, 3.4, 100],
    [-1.5, 1, 100],
    [-2, -2, 300],
    [-1.3, 3.2, 100],
    [-2.1, 1.5, 100],
    [-3.8, 3, 100],
    [-0.3, 2.5, 100],
])

et_pos_array = np.zeros(np.shape(et_array))
et_pos_array[:,:2], et_time_array = et_array[:,:2], et_array[:,2]


# calculates angle of vector v
def polar_angle(v):
    return (np.arctan2(v[1], v[0]) + 2*PI) % (2*PI)


# takes polar angle and returns bin index and binned angle
def bin_angle(angle):
    for i, bin in enumerate(np.linspace(0, 2*PI, bin_count+1)):
        if bin > (angle + 2*PI) % (2*PI) - PI/bin_count:
            return i%8, bin

In [76]:
%%capture_video --path "animations/eye_tracking/eye_tracking_buff.mp4"
%%manim -qh --fps 60 $video_scene

from cmath import sqrt
from turtle import fillcolor


class et_Scene(Scene):
    def construct(self):
        #self.camera.background_color = WHITE    
        pol = PolarPlane(
            azimuth_units = "degrees", 
            size = 4, 
            azimuth_label_font_size = 33.6, 
            azimuth_step = bin_count, 
            azimuth_label_buff = 0.2,
            ).add_coordinates(r_values = []).shift(4*RIGHT)
        self.add(pol)
        phi_vals = np.linspace(0, 2*PI, bin_count+1)
        r_vals = np.zeros_like(phi_vals)

        angle_list = [polar_angle(et_pos_array[i] - et_pos_array[i-1]) for i in range(1, len(et_pos_array))]
        dot_list = [Dot(color = BLUE, radius = 0.1).move_to(p) for p in et_pos_array]
        arrow_list = [Line(start = et_pos_array[i-1], end = et_pos_array[i], color = RED, stroke_width = 1.5, buff = 0.2).add_tip(tip_length = 0.15, tip_width = 0.15) for i in range(1, len(et_pos_array))]
        arc_base_list = [DashedLine(start = et_pos_array[i-1], end = et_pos_array[i-1] + [npl.norm(et_pos_array[i] - et_pos_array[i-1]) / 2, 0, 0]) for i in range(1, len(et_pos_array))]
        arc_list = [Arc(radius = npl.norm(et_pos_array[i] - et_pos_array[i-1]) / 2, arc_center = et_pos_array[i-1], angle = angle_list[i-1]) for i in range(1, len(et_pos_array))]
        arc_half_end_list = [Arc(radius = npl.norm(et_pos_array[i]- et_pos_array[i-1]) / 2, arc_center = et_pos_array[i-1], angle = angle_list[i-1]/2).get_end() for i in range(1, len(et_pos_array))]
        arc_text_list = [Tex(f'{round(angle_list[i-1]/2/PI*360)}'r'$\,^{\circ}$').next_to(arc_list[i-1], buff = 0.1, direction = arc_half_end_list[i-1] - et_pos_array[i-1]) for i in range(1, len(et_pos_array))]
        binned_angel_list = [bin_angle(angle_list[i-1]) for i in range(1, len(et_pos_array))]

        add = 1
        for i, pos_next in enumerate(et_pos_array):
            
            if i == 0:
                x_vals = [r_vals[i] * np.cos(phi_vals[i]) for i in range(len(phi_vals))]
                y_vals = [r_vals[i] * np.sin(phi_vals[i]) for i in range(len(phi_vals))]  

                pos_list = np.array([[x_vals[i], y_vals[i], 0] for i in range(len(x_vals))]) / 2
                poly0 = Polygon(*pos_list, fill_color = YELLOW, fill_opacity = 0.2, color = YELLOW).shift(4*RIGHT) 

                graph0 = pol.plot_line_graph(x_values = x_vals, y_values = y_vals, stroke_color = YELLOW, add_vertex_dots = False, stroke_opacity = 0.5)        
                self.add(graph0)
                self.play(FadeIn(dot_list[i]))
            else:
                #end_gaze_group.add(dot_list[i-1], arrow_list[i-1])
                pos = et_pos_array[i-1]
                angle = polar_angle(pos_next - pos)
                index, binned_angel = bin_angle(angle)

                r_vals[index] += add
                r_vals[-1] = r_vals[0] 

                if max(r_vals) > 3:
                    r_vals = r_vals / max(r_vals) * 3.5
                    add = add / max(r_vals) * 3.5

                x_vals = [r_vals[i] * np.cos(phi_vals[i]) for i in range(len(phi_vals))]
                y_vals = [r_vals[i] * np.sin(phi_vals[i]) for i in range(len(phi_vals))]

                plot_line = Line(start = [0, 0, 0], end = [r_vals[index] * np.cos(binned_angel) / 2, r_vals[index] * np.sin(binned_angel) / 2, 0], color = RED, buff = 0).shift(4*RIGHT)

                # smoothing
                phi_smooth = np.linspace(min(phi_vals), max(phi_vals), 500)
                spl = make_interp_spline(phi_vals, r_vals, k=2) 
                r_smooth = spl(phi_smooth)

                x_smooth = [r_smooth[i] * np.cos(phi_smooth[i]) for i in range(len(phi_smooth))]
                y_smooth = [r_smooth[i] * np.sin(phi_smooth[i]) for i in range(len(phi_smooth))]

                graph = pol.plot_line_graph(x_values = x_vals, y_values = y_vals, stroke_color = YELLOW, add_vertex_dots = False, stroke_opacity = 1)
                graph_smooth = pol.plot_line_graph(x_values = x_smooth, y_values = y_smooth, stroke_color = YELLOW, add_vertex_dots = False, stroke_opacity = 1)

                # polygons for filled plot
                pos_list = np.array([[x_vals[i], y_vals[i], 0] for i in range(len(x_vals))]) / 2
                pos_smooth_list = np.array([[x_smooth[i], y_smooth[i], 0] for i in range(len(x_smooth))]) / 2

                poly = Polygon(*pos_list, fill_color = YELLOW, fill_opacity = 0.2, color = YELLOW).shift(4*RIGHT)
                poly_smooth = Polygon(*pos_smooth_list, fill_color = YELLOW, fill_opacity = 0.2, color = YELLOW).shift(4*RIGHT)
                if i < 5:
                    if i < 4:
                        self.play(FadeIn(dot_list[i]), Create(arrow_list[i-1]), Create(arc_list[i-1]), Create(arc_base_list[i-1]), Write(arc_text_list[i-1]))
                        self.wait(0.5)
                        self.play(TransformFromCopy(arrow_list[i-1], plot_line), Transform(poly0, poly))#, Transform(graph0, graph))# 
                        self.play(FadeOut(plot_line))
                        self.play(Uncreate(arc_list[i-1]), Uncreate(arc_base_list[i-1]), Uncreate(arc_text_list[i-1]), run_time = 0.75)
                    else:
                        tbr = VGroup(dot_list[i-4], arrow_list[i-4])
                        self.play(FadeIn(dot_list[i]), Create(arrow_list[i-1]), Create(arc_list[i-1]), Create(arc_base_list[i-1]), Write(arc_text_list[i-1]), Uncreate(tbr))
                        self.wait(0.5)
                        self.play(TransformFromCopy(arrow_list[i-1], plot_line), Transform(poly0, poly))#, Transform(graph0, graph))# 
                        self.play(FadeOut(plot_line))
                        self.play(Uncreate(arc_list[i-1]), Uncreate(arc_base_list[i-1]), Uncreate(arc_text_list[i-1]), run_time = 0.75)          
                elif i < 10:
                    tbr = VGroup(dot_list[i-4], arrow_list[i-4])
                    self.play(FadeIn(dot_list[i]), Create(arrow_list[i-1]), Uncreate(tbr), run_time = 0.5)
                    self.play(TransformFromCopy(arrow_list[i-1], plot_line), Transform(poly0, poly), run_time = 0.5)#, Transform(graph0, graph))#
                    self.play(FadeOut(plot_line), run_time = 0.5)
                else:
                    tbr = VGroup(dot_list[i-4], arrow_list[i-4])
                    self.play(FadeIn(dot_list[i]), Create(arrow_list[i-1]), Create(plot_line), Transform(poly0, poly), Uncreate(tbr), run_time = 0.15)
                    self.play(FadeOut(plot_line), run_time = 0.15)
                    
        #self.play(Transform(graph0, graph_smooth))
        tbr2 = VGroup(dot_list[i-3], dot_list[i-2], dot_list[i-1], dot_list[i], arrow_list[i-4], arrow_list[i-3], arrow_list[i-2], arrow_list[i-1])
        self.play(FadeOut(tbr2))
        dot_list2 = [Dot(color = BLUE, radius = 0.1).move_to(p) for p in et_pos_array]
        arrow_list2 = [Line(start = et_pos_array[i-1], end = et_pos_array[i], color = RED, stroke_width = 1.5, buff = 0.2).add_tip(tip_length = 0.15, tip_width = 0.15) for i in range(1, len(et_pos_array))]
        end_gaze_group = VGroup(dot_list2[0])
        for i in range(1, len(dot_list2)):
            end_gaze_group.add(arrow_list2[i-1], dot_list2[i])
        self.play(Transform(poly0, poly_smooth), FadeIn(end_gaze_group), run_time = 2)
        self.wait(3)